In [ ]:
import torch
import ujson
import numpy as np
import os.path
import random
import urllib
import zipfile
import os
import ssl
from torch.utils.data import Dataset, DataLoader
import torch
import importlib
from torchvision import datasets, transforms
import torch.nn as nn
from tqdm import tqdm

from benchmark.cifar10.model.cnn import Model

In [ ]:
rawdata_path = "/mnt/disk1/tiennh/provably_fl/benchmark/cifar10/data"
dist_id = 9
skewness = 0.1 if dist_id != 0 else 0
num_clients = 100
minvol = 10
seed = 0
num_classes = 10

In [ ]:
train_data = datasets.CIFAR10(rawdata_path, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]))
test_data = datasets.CIFAR10(rawdata_path, train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]))

In [ ]:
seed = 0
batch_size = 128
torch.manual_seed(seed)

model = Model()
criterion = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.05)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

num_epochs = 200
device = "cuda:0" if torch.cuda.is_available() else "cpu"


model = model.to(device)
model.load_state_dict(torch.load("./checkpoint/adv_model (1).pth"))

In [ ]:
from algorithm.fedprob_utils.smooth import Smooth
from tqdm import tqdm

smooth_model = Smooth(model, 10, 0.25, 100, 1000, 0.05, device, 64)
results = []
for inputs, outputs in tqdm(test_data):
    inputs = inputs.to(device)
    preds, radius = smooth_model.certify(inputs)
    results.append({
        "correct": outputs == preds,
        "radius": radius
    })

In [ ]:
import numpy as np
import pandas as pd
from algorithm.fedprob_utils.accuracy import ApproximateAccuracy

test_radii = np.arange(0, 1.6, 0.1)
certify_results = pd.DataFrame(results)
accuracy_calculator = ApproximateAccuracy(certify_results)
centralize_cert_accs = accuracy_calculator.at_radii(test_radii)


In [ ]:
server_cert_accs = [0.6292, 0.5967, 0.5589, 0.5171, 0.4747, 0.4323, 0.3858, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(test_radii, centralize_cert_accs, label="Centralize")
plt.plot(test_radii, server_cert_accs, label="FL server")
plt.legend()
plt.show()

In [ ]:
sample_data = train_data[0][0], train_data[0][1]
num_synthetic = 64
synthetic_sample = torch.randn([num_synthetic, 3, 32, 32], requires_grad=True)
synthetic_label = torch.empty(num_synthetic, dtype=torch.long).random_(10).to(device)

synth_optim = torch.optim.SGD([synthetic_sample], lr=0.1)
synth_criterion = nn.CrossEntropyLoss()

synth_epochs = 1000

In [ ]:
for epoch in tqdm(range(synth_epochs)):
    model.zero_grad()
    synthetic_sample.grad.zero_()
    inputs = synthetic_sample.to(device)
    outputs = model(inputs)
    loss = synth_criterion(outputs, synthetic_label)
    loss.backward()
    synth_optim.step()

In [4]:
%%sh
{
    task_name="cifar10_cnum100_dist0_skew0_seed0"
    learning_rate=0.05
    sigma=0.25
    model="cnn"
    fedalg="fedprob_v2"
    num_round=200
    CUDA_VISIBLE_DEVICES=0 python main.py --sigma $sigma --task $task_name --model $model --algorithm $fedalg --num_rounds $num_round --num_epochs 5 --learning_rate $learning_rate --proportion 0.1 --batch_size 10 --eval_interval 1
}

Training Loss:                0.0519
Testing Loss:                 1.9463
Testing Accuracy:             0.6258
Validating Accuracy:          0.6233
Mean of Client Accuracy:      0.6233
Std of Client Accuracy:       0.0533
--------------Round 166--------------
Time Cost:                    7.0818s
Training Loss:                0.0567
Testing Loss:                 1.8666
Testing Accuracy:             0.6230
Validating Accuracy:          0.6203
Mean of Client Accuracy:      0.6203
Std of Client Accuracy:       0.0534
--------------Round 167--------------
Time Cost:                    7.0902s
Training Loss:                0.0488
Testing Loss:                 1.9616
Testing Accuracy:             0.6255
Validating Accuracy:          0.6229
Mean of Client Accuracy:      0.6229
Std of Client Accuracy:       0.0535
--------------Round 168--------------
Time Cost:                    7.0859s
Training Loss:                0.0477
Testing Loss:                 1.9743
Testing Accuracy:             0.